In [7]:
import sys
import os
# add parent directory (where "natsume" lives) to sys.path so I can import natsume
sys.path.append(r"C:\Users\WBS\Desktop\EXOPLANET WORK\006 NATSUME")
import natsume

import ttv_curvefit.curvefit as ttvfit  # custom-written
import ttvfast
from ttvfast import models

from joblib import Parallel, delayed
from tqdm import tqdm
import pandas as pd
import numpy as np
from astropy import units as u
from astropy.constants import M_earth
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter

In [8]:
# Create 10000 samples for TTVFast for each MMR
size = 10000 * 10
mu1 = 3e-6; mu2 = 3e-6
e1 = 10**np.random.uniform(low=-3, high=-0.5, size=size); e2 = e1.copy()
w1 = np.random.uniform(low=0, high=360, size=size)
w2 = (w1.copy() + np.random.choice([0, 180], size=size)) % 360

# MMR sampling
mmr_list = ['2:1', '3:2', '4:3', '5:4', '3:1', '5:3', '7:5', '4:1', '5:2', '5:1']
mmr = np.random.choice(mmr_list, size=size)
mmrparts = np.char.split(mmr, ':')  # Extract j:j-N
j = np.array([int(p[0]) for p in mmrparts])
N = np.array([int(p[0]) - int(p[1]) for p in mmrparts])

# Dynamic Delta sampling
mmr_conds = [(mmr == ratio) for ratio in mmr_list]
Delta_maxes = [0.1, 0.1, 0.05, 0.05,
               0.1, 0.05, 0.02, 0.1, 0.05, 0.05]
Delta_high = np.select(mmr_conds, [mx for mx in Delta_maxes], default=0.05)

Delta = np.random.uniform(low=-Delta_high, high=Delta_high, size=size)

# P2 calculation
P1 = 30; P2 = P1 * (Delta + 1) * j / (j-N)

# Compile to dataframe
df = pd.DataFrame({
    'e1': e1, 'w1': w1,
    'e2': e2, 'w2': w2,
    'Delta': Delta,
    'mmr': mmr, 'j': j, 'N': N,
    'P1': P1, 'P2': P2,
    'mu1': mu1, 'mu2': mu2,
    'valid': True
})
df.head()

,e1,w1,e2,w2,Delta,mmr,j,N,P1,P2,mu1,mu2,valid
0,0.001410,214.983207,0.001410,34.983207,0.092250,3:2,3,1,30,49.151241,0.000003,0.000003,True
1,0.089057,319.762149,0.089057,319.762149,-0.017099,7:5,7,2,30,41.281823,0.000003,0.000003,True
2,0.026567,33.578327,0.026567,213.578327,-0.015354,4:3,4,1,30,39.385850,0.000003,0.000003,True
3,0.036570,53.952120,0.036570,233.952120,0.076126,3:1,3,2,30,96.851375,0.000003,0.000003,True
4,0.004234,32.758994,0.004234,32.758994,0.025265,4:3,4,1,30,41.010602,0.000003,0.000003,True


In [9]:
# Copy dataframe for fitting results
df_fit = df.copy()
df_fit_cols = ["A1", "B1", "V1", "Pttv1", "phase1", "std1", "res1",
               "A2", "B2", "V2", "Pttv2", "phase2", "std2", "res2",
               "R2_1", "R2_2"]
df_fit[df_fit_cols] = np.nan
df_fit.head()

,e1,w1,e2,w2,Delta,mmr,j,N,P1,P2,...,res1,A2,B2,V2,Pttv2,phase2,std2,res2,R2_1,R2_2
0,0.001410,214.983207,0.001410,34.983207,0.092250,3:2,3,1,30,49.151241,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.089057,319.762149,0.089057,319.762149,-0.017099,7:5,7,2,30,41.281823,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.026567,33.578327,0.026567,213.578327,-0.015354,4:3,4,1,30,39.385850,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.036570,53.952120,0.036570,233.952120,0.076126,3:1,3,2,30,96.851375,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.004234,32.758994,0.004234,32.758994,0.025265,4:3,4,1,30,41.010602,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
def ttv_model(t, A, B, V, Pttv, phase):
    return A + B*t + V * np.sin(2*np.pi/Pttv * t + phase)

In [11]:
# TTVFast simulation (Copypasted from 1st order code)
# Create TTV simulations and extract Amplitude and Period
gravity = 0.000295994511 # AU^3/day^2/M_sun
stellar_mass = 1.0       # M_sun

for i, row in tqdm(df.iterrows(), total=len(df)):  
    # Initialize variables
    P1 = row['P1']; P2 = row['P2']
    j = row['j']; N = row['N']
    PTTV = 1/np.abs(j/P2 - (j-N)/P1)
    
    e1 = row['e1']; w1 = row['w1']
    e2 = row['e2']; w2 = row['w2']
    mu1 = row['mu1']; mu2 = row['mu2']
    
    # Create TTVFast planets
    planet1 = models.Planet(
        mass=mu1,            # M_sun
        period=P1,           # days
        eccentricity=e1,
        inclination=90,      # degrees
        longnode=0,          # degrees
        argument=90+w1,      # degrees
        mean_anomaly=0       # degrees
    )
    planet2 = models.Planet(
        mass=mu2,
        period=P2,
        eccentricity=e2,
        inclination=90,
        longnode=0,
        argument=90+w2,
        mean_anomaly=0,
    )
    planets = [planet1, planet2]
    
    begin_time = -P1 * 50
    Time = begin_time  # days
    dt = P1 / 100       # days
    Total = min(1600, PTTV * 2.5)   # days
    
    results = ttvfast.ttvfast(planets, stellar_mass, Time, dt, Total,
                              input_flag=1)

    # Create TTV signal from TTVFast Output
    out = pd.DataFrame(results['positions']).transpose()
    out.columns = ['planet', 'epoch', 'times', 'rsky', 'vsky']
    
    table0 = out[(out.planet == 0) & (out.times > 0)]
    n0 = table0.epoch
    tn0 = table0.times

    table1 = out[(out.planet == 1) & (out.times > 0)]
    n1 = table1.epoch
    tn1 = table1.times

    # If few TTVFast output, skip
    len_threshold = 10
    if (len(n0) <= len_threshold) or (len(n1) <= len_threshold):
        continue
        
    time0, ttv0, ttv0_err = ttvfit.return_ttv(epochs=n0, midtransits=tn0)
    time1, ttv1, ttv1_err = ttvfit.return_ttv(epochs=n1, midtransits=tn1)
    
    # Make sure ttv0 and ttv1 actually are centered around O-C = 0
    ttv0 = ttv0 - (np.max(ttv0) + np.min(ttv0)) / 2
    ttv1 = ttv1 - (np.max(ttv1) + np.min(ttv1)) / 2
    
    # Get sinusoidal TTV model parameters
    popt_ttv0, perr_ttv0, r2_0 = ttvfit.characterize_ttv(time=time0, ttv=ttv0, ttv_err=ttv0_err,
                                                         expected_period=PTTV, method='lmfit')
    popt_ttv1, perr_ttv1, r2_1 = ttvfit.characterize_ttv(time=time1, ttv=ttv1, ttv_err=ttv1_err,
                                                         expected_period=PTTV, method='lmfit')
    std0, std1 = np.std(ttv0), np.std(ttv1)  # TTV dispersion
    res0, res1 = np.std(ttv0 - ttv_model(time0, *popt_ttv0)), np.std(ttv1 - ttv_model(time1, *popt_ttv1)) # Residual dispersion

    # Write results to df_fit
    df_fit.loc[i, df_fit_cols]  = [*popt_ttv0, std0, res0, *popt_ttv1, std1, res1, r2_0, r2_1]

100%|████████████████████████████████████████████████████████████████████████| 100000/100000 [2:17:43<00:00, 12.10it/s]


In [12]:
# If output fit has no values, convert valid from TRUE to FALSE
output_has_nan = df_fit[df_fit_cols].isna().any(axis=1)
df_fit.loc[output_has_nan, 'valid'] = False

# If output has PTTV wildly different from each other, convert valid from TRUE to FALSE
Pttv_ratio_threshold = 1.01
Pttv_not_similar = (df_fit['Pttv1'] / df_fit['Pttv2'] > Pttv_ratio_threshold) | (df_fit['Pttv2'] / df_fit['Pttv1'] > Pttv_ratio_threshold)
df_fit.loc[Pttv_not_similar, 'valid'] = False

df_fit.head()

,e1,w1,e2,w2,Delta,mmr,j,N,P1,P2,...,res1,A2,B2,V2,Pttv2,phase2,std2,res2,R2_1,R2_2
0,0.001410,214.983207,0.001410,34.983207,0.092250,3:2,3,1,30,49.151241,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.089057,319.762149,0.089057,319.762149,-0.017099,7:5,7,2,30,41.281823,...,0.000525,-6.656430e-05,1.899893e-07,0.000589,333.690662,0.592429,0.000791,0.000672,0.273060,0.278562
2,0.026567,33.578327,0.026567,213.578327,-0.015354,4:3,4,1,30,39.385850,...,0.001932,3.852556e-04,-2.010537e-07,0.008393,652.399372,4.865618,0.006332,0.002289,0.870769,0.869298
3,0.036570,53.952120,0.036570,233.952120,0.076126,3:1,3,2,30,96.851375,...,0.000013,-1.178645e-07,-4.106038e-10,0.000059,424.310264,2.665432,0.000041,0.000001,0.712373,0.999249
4,0.004234,32.758994,0.004234,32.758994,0.025265,4:3,4,1,30,41.010602,...,0.000504,7.729594e-05,-1.330196e-07,0.000925,401.978009,3.858321,0.000902,0.000629,0.448899,0.514417


In [13]:
# Export Outputs
df_fit.to_csv("allorder_TTVFast_fits.csv", index=False)